<a href="https://colab.research.google.com/github/apparenelle/3D-Web/blob/main/Project_Valeria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Valeria (Algo Trading Bot)

Updates:
- i made progress. we can view unusual daily volume for nvda

**Libraries**
- yfinance - provides stock price history | Open Close High Low Volume Stock_Splits
- datetime - standard datetime obj.
- numpy - statistical analysis | percentile
- math - cleaning numerics | ceil

In [9]:
import yfinance as yf
import datetime
import numpy as np
import math

Apple Stock Info

History of AAPL  for the last 2 years || 2 years ago was the last stock split

- to allow 30 years of data analysis, adjust all stock data for stock splits

In [10]:
# Find and print all days where volume is higher than 87 million

In [11]:
def printDaysOfHighVolume_ARR(X, Y, target_percentile):
  arr = list()
  for x,y in zip(X, Y):
    if (y > target_percentile): #replace 115 mil w/ the target above and convert this to a function
      print(x, 'Volume:  ' + str(y))
      arr.append([x,y])
  return arr

In [12]:
# printDaysOfHighVolume_ARR(aapl_date_vol_df['Date'], aapl_date_vol_df['Volume'], stock_AT_80_percentile)

Some months above have continous days of large orders in the same week

In [13]:
# good for managing stock info
class stock_data:
  def __init__(self, symbol:str):
    self.symbol = symbol
    self.stock = yf.Ticker(symbol)
    self.info = self.stock.info

  def getHistory(self, days):
    symbol = self.symbol
    stock = self.stock
    history = stock.history(period=(str(days)+'d'), interval='1d').reset_index()
    self.history = history
    return history

  def calcVolumePercentilesFromHist(self):
    history = self.history

    # retrieves 70th, 80th, and 90th, percentile
    if history.empty is False:
      percentiles = {
                      '70': math.ceil(np.percentile(self.history['Volume'], 70)),
                      '80': math.ceil(np.percentile(self.history['Volume'], 80)),
                      '90': math.ceil(np.percentile(self.history['Volume'], 90))
                    }
      self.vol_percentiles = percentiles
      return percentiles
    else:
      print('History is empty')
      print('Please call getHistory()')
      return None

  def findLastStockSplitIndex(self):
    history = self.history

    if not history.empty:
      for index, value in history['Stock Splits'][::-1].items():
        if value > 0.0:
          return index
      return 'No stock splits present.'

  def movingAverages(self):
    #Grab history
    history = self.history

    # Add columns for 5-Day, 10-Day, and 20-Day moving averages
    history['5-Day_Moving_Avg'] = history['Close'].rolling(window=5).mean()
    history['10-Day_Moving_Avg'] = history['Close'].rolling(window=10).mean()
    history['20-Day_Moving_Avg'] = history['Close'].rolling(window=20).mean()

    #Save to object df
    self.history = history

  def removeTime(self):
    # removes time from stock history for cleaner code
    history = self.history

    if history.empty is False:
      def dateColtoDateObj(a):
        return a.date()

      # time mentioned in Date col. is removed;
      # self.last_history['Date'] = list(map(dateColtoDateObj, history['Date']))

      # or
      self.history['Date'] = self.history['Date'].apply(lambda a: a.date())

      return self.history
    else:
      print('History never was saved')
      print('Please recover history.')
      return None



In [14]:
#pre-processing for AI models
class data_cleaner:
  def __init__(self, data, stock):
    self.data = data
    self.stock = stock

  def display_info(self):
    print(self.data)

  def clean(self):
    dataframe = self.data
    stock = self.stock

    #all yf data already adjusted for splits
    dataframe = dataframe.drop(columns=['Stock Splits', 'Dividends'], axis=1)

    # stock percentiles or 'low' make out of 0 - 4
    def rankingVolume(a, ranking):
      if a <= ranking['70']:
        return 0
      elif a > ranking['70'] and a <= ranking['80']:
        return 1
      elif a > ranking['80'] and a <= ranking['90']:
        return 2
      elif a >= ranking['90']:
        return 3
      else:
        print('Something is off w/ the volume')
        return -1

    # Adds volume ranking to df
    dataframe['Volume_Ranking'] = dataframe['Volume'].apply(lambda x: rankingVolume(x, stock.vol_percentiles))


    def priceMovement(row):
      low = row['Low']
      high = row['High']
      open_ = row['Open']  # Renamed to open_ to avoid conflict with Python's open function
      close = row['Close']

      if close >= open_:
        return high - low
      else:
        return (high - low) * -1

    #Price change from open to close
    dataframe['Price_Movement'] = dataframe.apply(priceMovement, axis=1)

    #add a day column for an iterator called days
    dataframe['Day'] = range(1, len(dataframe)+1)


    #what is the measure of success?

    #saves changes
    self.data = dataframe

  def reOrder(self):
    new_order = ['Date', 'Day', 'Price_Movement', 'Open', 'High', 'Low', 'Close', 'Volume', 'Volume_Ranking', '5-Day_Moving_Avg', '10-Day_Moving_Avg', '20-Day_Moving_Avg']
    self.data = self.data[new_order]
    return self.data

  def toCSV(self, fileName, sep):
    data = self.data
    data.to_csv(fileName, sep=sep, index=False, encoding='utf-8')


In [15]:
def main1():
  # relevant_fields = ['Open', 'High', 'Low', 'Close', 'Volume', 'Stock Split']

  stock = stock_data('aapl')

  stock.getHistory(days=1826)
  stock.removeTime()

  stock.movingAverages()
  stock.calcVolumePercentilesFromHist()

  # print(stock.history)
  # print(stock.calcVolumePercentilesFromHist())

  #cleaner must be used after stock attributes have been configured
  cleaner = data_cleaner(stock.history, stock)
  cleaner.clean()

  cleaner.reOrder()
  cleaner.display_info()

  cleaner.toCSV(fileName='./stock_data.csv', sep=',')


  # Relevant Fields
  # ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Split', 'Volume Ranking', 'Price Movement', '5-Day_Moving_Avg', '10-Day_Moving_Avg', '20-Day_Moving_Avg']

  # 70th Percentile         87 million shares a day  --- AAPL
  # 80th Percentile         96 million shares a day  --- AAPL
  # 90th Percentile         115 million shares a day --- AAPL

  # RETURNING processed dataset
  return cleaner.data


In [16]:
main1()

            Date   Day  Price_Movement        Open        High         Low  \
0     2016-11-11     1        0.539088   24.891006   25.297645   24.758557   
1     2016-11-14     2       -0.866723   25.028099   25.051335   24.184613   
2     2016-11-15     3        0.353195   24.763202   25.021128   24.667933   
3     2016-11-16     4        0.843488   24.793407   25.613659   24.770171   
4     2016-11-17     5        0.353195   25.516068   25.641546   25.288351   
...          ...   ...             ...         ...         ...         ...   
1821  2024-02-09  1822        1.990005  188.649994  189.990005  188.000000   
1822  2024-02-12  1823       -1.880005  188.419998  188.669998  186.789993   
1823  2024-02-13  1824       -2.700012  185.770004  186.210007  183.509995   
1824  2024-02-14  1825       -3.089996  185.320007  185.529999  182.440002   
1825  2024-02-15  1826        3.090012  183.550003  184.490005  181.399994   

           Close     Volume  Volume_Ranking  5-Day_Moving_Avg  

,Date,Day,Price_Movement,Open,High,Low,Close,Volume,Volume_Ranking,5-Day_Moving_Avg,10-Day_Moving_Avg,20-Day_Moving_Avg
0,2016-11-11,1,0.539088,24.891006,25.297645,24.758557,25.195404,136575600,1,NaN,NaN,NaN
1,2016-11-14,2,-0.866723,25.028099,25.051335,24.184613,24.563368,204702000,3,NaN,NaN,NaN
2,2016-11-15,3,0.353195,24.763202,25.021128,24.667933,24.888680,129058000,1,NaN,NaN,NaN
3,2016-11-16,4,0.843488,24.793407,25.613659,24.770171,25.557890,235362000,3,NaN,NaN,NaN
4,2016-11-17,5,0.353195,25.516068,25.641546,25.288351,25.548599,110528000,0,25.150788,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2024-02-09,1822,1.990005,188.649994,189.990005,188.000000,188.850006,45155200,0,188.519644,187.828426,188.921979
1822,2024-02-12,1823,-1.880005,188.419998,188.669998,186.789993,187.149994,41781900,0,188.461481,187.394859,188.995325
1823,2024-02-13,1824,-2.700012,185.770004,186.210007,183.509995,185.039993,56529500,0,187.657727,187.118823,189.077526
1824,2024-02-14,1825,-3.089996,185.320007,185.529999,182.440002,184.149994,54630500,0,186.654001,187.117323,189.162666


In [17]:
def main2():
  data = main1()



Good News: The data is auto adjusted by yahoo to account for stock splits no work necessary and you can train on as much data as I please!

*Tips*
```
# This is formatted as code
# stock = stock_data('aapl')
# stock.getHistory(1000)
# stock.removeTime()
# stock.history
# stock.info
# stock.calcVolumePercentilesFromHist()


# stock.history.iloc[147]
# stock.history.loc[147][:]
# cleaner = data_cleaner(stock.history, stock)
# stock.history.keys()
# cleaner.display_info()
```